In [1]:
#  word level CTC  tensorflow version 1.3

# this module requires the following :

#  a dataset csv file ( have a look at dataset csv generator)
#  a dictionary.txt file ( have a look at dictionary folder)



# to do :   
#  , testing on realdataset , get rid of fixed batch size , instead use variable batch size  


# perform following tests :

# 1. variable batch sizes test
# 2. GPU test





# please change the following according to your system
# hyperparameters in this notebook

# change paths in this notebook to that corresponding to your system
# please note number of audio examples in dataset must be even 

# peace.




In [2]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tensorflow.contrib.data import Dataset, Iterator
import time


/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
inputs = audio_to_mfcc('data/hardik_500/5.wav')

In [5]:
inputs.shape

(449, 13)

In [6]:
def decode_csv(line):
       parsed_line = tf.decode_csv(line, [[""],[""]])
       
    
       
       

       return parsed_line[0] , parsed_line[1]

In [7]:
# we also need a fixed vocabulary 
import re

word_dictionary = {}

with open("words/hardik_words.txt") as file:
    for i , line in enumerate(file):
        
        line = line.replace("\n", "")
        word_dictionary[line] = i



In [8]:
print(len(word_dictionary))

1303


In [9]:



def word_to_index(sentence):
   
    words = sentence.split(' ')
    index_list=[]
    for word in words:
       
        if word in word_dictionary:
           # print(word)
            index_list.insert(len(index_list) , word_dictionary[word])
            index_list.insert(len(index_list) , word_dictionary[' '])
    index_list.pop()        
    return index_list
  

In [10]:
word_to_index('I don't wanna say')

SyntaxError: invalid syntax (<ipython-input-10-3eb6f43c5fd7>, line 1)

In [11]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]

In [12]:
def _read_py_function(audio, label):
    audio = audio_to_mfcc(audio)
    
    if audio.shape[0] < timesteps:
        original_length=audio.shape[0]
       # print(original_length)
    
    elif audio.shape[0] >= timesteps:
        original_length=timesteps
        
    
    audio=pad(audio)
   
    return audio ,label, original_length

In [13]:
dataset = tf.contrib.data.TextLineDataset("hardik2.csv")
dataset=dataset.map(decode_csv)



dataset = dataset.map(
    lambda audio, label: tuple(tf.py_func(
        _read_py_function, [audio, label], [tf.double, label.dtype, tf.int64])))




dataset=dataset.batch(4)


Instructions for updating:
Use `tf.data.TextLineDataset`.


In [14]:

iterator = dataset.make_initializable_iterator()

item = iterator.get_next()

In [15]:
# this function is required for CTC Loss
# for it's input , first convert transcrition / ground truth to number representation 

def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape

In [16]:
# Training Parameters
learning_rate = 0.01
training_steps = 100
#batch_size = 2
display_step = 200
num_features = 13
logs_path = '/home/saurabh/Documents/tf_orange/tf_orange/hardik/logs'
model_path = '/home/saurabh/Documents/tf_orange/tf_orange/models/hardik/model_500.ckpt'

# Network Parameters

timesteps = 449 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = len(word_dictionary) + 1 # words.txt , all words plus space word  ( 10001) + CTC symbol (1)




In [17]:
#prediction = tf.nn.softmax(logits)
inputs = tf.placeholder(tf.float32, [None, None, num_features])
targets = tf.sparse_placeholder(tf.int32)
seq_len = tf.placeholder(tf.int32, [None])
batch_size = 4

#batch_size = tf.placeholder ( tf.int32 )


In [20]:
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(500):
      value = sess.run(item[1])
      print(i)
      print(value)

0
[b'Flexible battery inspired by human spine'
 b'Amateur radio astronomer discovers long-lost satellite'
 b'community members visit India' b'Moderate Blood Sugar Levels']
1
[b'health benefits of pear consumption' b'Fell Asleep With Headache'
 b'Woke Up With A British Accent' b"Don't want to diet?"]
2
[b'lose weight by following this simple trick'
 b'Diarrhoea outbreak in Kolkata'
 b"A concise look at Darwin's Theory of Evolution and its criticism"
 b'tax you have to pay']
3
[b'Image Makeover Under ' b'100 Per Cent Placement'
 b'Midcap continues to outperform' b"boost India's energy security"]
4
[b"India's ultra rich spend?" b'restructured accounts will maintain'
 b'board accepts resignation' b'Riding on regulatory win']
5
[b'Scariest Flight Of My Life' b'bet a million on you'
 b'no way around this' b"only way now it's through"]
6
[b'gonna cry myself to sleep' b'kept you to winter'
 b"you won't be needing me" b'not in front of me']
7
[b'catch your tears' b'leave me this way' b'A puddle

68
[b'but shoot so high' b'So much life in those' b'you look for the light'
 b'when your wounds open']
69
[b"'HTC U12' With 5G Support" b'you will cry' b'I can see a rainbow'
 b'they fall on down']
70
[b'see your soul grow' b'Through the pain' b'they hit the ground'
 b'As the sun comes out']
71
[b'Through the storms' b'you cannot escape' b'You can do it'
 b'Renault Kwid Superhero Edition']
72
[b'just feel baby' b'turn the radio on'
 b"Friday night and I won't be long" b'Gotta do my hair']
73
[b'I hit the dance floor' b'I got all I need'
 b'dollar bills to have fun tonight' b'I love cheap thrills']
74
[b'I can feel the beat' b'God shaped hole leaves' b'iBall CompBook Premio'
 b"friends won't bring relief"]
75
[b'Drown your voice with party' b'For one and only'
 b'Faith creeps slowly over me' b'Took twelve steps']
76
[b'And carried me' b'Resurrection on me' b'Blunted thorns are soft'
 b'weight of life lift me']
77
[b'Hear the chatter fooling me'
 b'Google Flips the Switch on Its Pixel Vi

UnimplementedError: Unsupported object type NoneType
	 [[Node: PyFunc = PyFunc[Tin=[DT_STRING, DT_STRING], Tout=[DT_DOUBLE, DT_STRING, DT_INT64], token="pyfunc_0"](arg0, arg1)]]
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[<unknown>, <unknown>, <unknown>], output_types=[DT_DOUBLE, DT_STRING, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator)]]

In [ ]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([  num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [ ]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
   # x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    

    
    # Get lstm cell output
   # outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell , x,
    #dtype=tf.float32)
    
    outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell,
                                   inputs=x,
                                   sequence_length=seq_len,
                                   dtype=tf.float32)
    
    
    #convert output shape (timesteps * batch * hidden units ) to (batch*timesteps* hidden units)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    outputs=tf.reshape(outputs, [-1,num_hidden])
    
  
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    res = tf.reshape(res, [batch_size,timesteps,num_classes])
    
    # new code 
    # state is of shape ( batch size ,  hidden_units)
    # need to adapt weights accordingly 
    
    
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])

In [ ]:




logits = RNN(inputs, weights, biases)

loss =  tf.nn.ctc_loss ( targets, logits , seq_len , time_major = False)
cost = tf.reduce_mean(loss)
optimizer = tf.train.MomentumOptimizer(learning_rate,
                                           0.9).minimize(cost)


# Option 2: tf.contrib.ctc.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
decoder_input = tf.transpose(logits, [1, 0, 2])

decoded, log_prob = tf.nn.ctc_greedy_decoder(decoder_input, seq_len)

    

In [ ]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [ ]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
# need this for decoding word give its index key value 

def keys_of_value(dct, value):
    for k in dct:
        if isinstance(dct[k], list):
            if value in dct[k]:
                return k
        else:
            if value == dct[k]:
                return k

In [ ]:
# testing above function 

print(keys_of_value(word_dictionary, 2419))

In [ ]:
# retrain from a checkpoint
# if training first time, ignore this block

with tf.Session() as sess:
    

    # Run the initializer
    sess.run(init)
        
    # Restore model weights from previously saved model
    load_path = saver.restore(sess, model_path +  '/2' )
    print("Model restored from file: %s" % load_path)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_steps):
     #   avg_cost = 0.
           # Save model weights to disk
        save_path = saver.save(sess, model_path +  '/2')
        print("Model saved in file: %s" % save_path) 
        
        
        sess.run(iterator.initializer)
        train_cost=0
        
        while True:
            try:
                Next_element=sess.run(item)
             
            
                 
                print(Next_element[1])   
                
                batch_list = []
                
 
                for index,j in enumerate(Next_element[1]):
                    batch_list.insert(index, word_to_index(j.decode("utf-8")))
                    
                
                batch_targets = sparse_tuple_from(batch_list)
                
                print(Next_element[2])
                
                feed = {inputs: Next_element[0],
                        targets: batch_targets,
                        seq_len: Next_element[2],
                         }
	    

                batch_cost, _ , summary = sess.run([cost, optimizer , merged_summary_op ], feed)
               # print(batch_cost)
                
                train_cost += batch_cost*batch_size
                
                
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch )
                
                 # Decoding
                d = sess.run(decoded[0], feed_dict=feed)
   
                dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=sess)
    
                for i, seq in enumerate(dense_decoded):

                    seq = [s for s in seq if s != -1]
       
                    str_decoded = ''.join([keys_of_value(word_dictionary, x) for x in seq ])
                    print(str_decoded)
            
              
            except tf.errors.OutOfRangeError:
              print(train_cost)
              
              
              break
                
        # Save model weights to disk at the end of all epochs, if we remove this last epoch doesnt gets saved
    save_path = saver.save(sess, model_path +  '/2')
    print("Model saved in file: %s" % save_path)        
    
    

In [ ]:
# Start training
with tf.Session() as sess:
    sess.run(init)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_steps):
     #   avg_cost = 0.
           # Save model weights to disk
        save_path = saver.save(sess, model_path)
        print("Model saved in file: %s" % save_path) 
        
        
        sess.run(iterator.initializer)
        train_cost=0
        
        while True:
            try:
                Next_element=sess.run(item)
             
            
                 
                print(Next_element[1])   
                
                batch_list = []
                
 
                for index,j in enumerate(Next_element[1]):
                    batch_list.insert(index, word_to_index(j.decode("utf-8")))
                    
                
                batch_targets = sparse_tuple_from(batch_list)
                
                print(Next_element[2])
                
                feed = {inputs: Next_element[0],
                        targets: batch_targets,
                        seq_len: Next_element[2],
                         }
	    

                batch_cost, _ , summary = sess.run([cost, optimizer , merged_summary_op ], feed)
               # print(batch_cost)
                
                train_cost += batch_cost*batch_size
                
                
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch )
                
                 # Decoding
                d = sess.run(decoded[0], feed_dict=feed)
   
                dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=sess)
    
                for i, seq in enumerate(dense_decoded):

                    seq = [s for s in seq if s != -1]
       
                    str_decoded = ''.join([keys_of_value(word_dictionary, x) for x in seq ])
                    print(str_decoded)
            
              
            except tf.errors.OutOfRangeError:
              print(train_cost)
              
              
              break
                
        # Save model weights to disk at the end of all epochs, if we remove this last epoch doesnt gets saved
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)         

               
        
            
                
                
             
                
                
            